Use PySpark to load all of the data

In [1]:
import pyspark

sc = pyspark.SparkContext()

In [2]:
raw = sc.textFile("../states/*.csv")

splitified = raw.filter(lambda x: x != "state,total_votes,dem_votes,dem_pct,dem_ev,rep_votes,rep_pct,rep_ev,year").map(lambda x: x.replace(",,", ",0,").replace(",*,", ",0,").replace("*", "").replace("--", "0")).map(lambda x: x.split(","))
formatted = splitified.map(lambda x: ((x[0], int(x[8])), (int(x[1]), int(x[2]), float(x[3]), int(x[4]), int(x[5]), float(x[6]), int(x[7]))))

print(splitified.takeSample(False, 5))
print(formatted.takeSample(False, 5))

[['Iowa', '1566031', '653669', '41.7', '0', '800983', '51.1', '6', '2016'], ['Michigan', '5664186', '2736533', '48.31', '0', '2816636', '49.73', '15', '2024'], ['New Jersey', '3611691', '1911430', '52.9', '15', '1670003', '46.2', '0', '2004'], ['Wisconsin', '3071434', '1620985', '52.9', '10', '1410966', '46.0', '0', '2012'], ['Connecticut', '1578769', '857488', '54.3', '7', '693826', '43.9', '0', '2004']]
[(('West Virginia', 1924), (583662, 257232, 44.1, 0, 288635, 49.5, 8)), (('Rhode Island', 1984), (410492, 197106, 48.0, 0, 212080, 51.7, 4)), (('Utah', 1952), (329554, 135364, 41.1, 0, 194190, 58.9, 4)), (('New Mexico', 1952), (238608, 105661, 44.3, 0, 132170, 55.4, 4)), (('Massachusetts', 2000), (2702984, 1616487, 59.8, 12, 878502, 32.5, 0))]


In [3]:
#ratio of dem, rep, and other
ratios = formatted.mapValues(lambda x: (x[1] / x[0], x[4] / x[0], (x[0]-x[4]-x[1]) / x[0]))

print(ratios.takeSample(False, 5))

alabama = ratios.filter(lambda x: x[0][0] == "Alabama").map(lambda x: (x[0][1], x[1]))

print(alabama.takeSample(False, 5))

[(('Vermont', 2000), (0.5063470921619528, 0.4069716079753184, 0.08668129986272884)), (('Rhode Island', 2012), (0.627009588632639, 0.35243661570813967, 0.020553795659221295)), (('Massachusetts', 1956), (0.4037417830740053, 0.5932269280981185, 0.003031288827876105)), (('Kansas', 2008), (0.41651967194013617, 0.5661225434349189, 0.01735778462494498)), (('South Dakota', 1932), (0.6362372502929572, 0.3439630007141916, 0.01979974899285115))]
[(1964, (0.0, 0.6945092763598515, 0.30549072364014856)), (1944, (0.8127627756462902, 0.18198681882627898, 0.005250405527430815)), (1984, (0.38280781265064545, 0.6054249354760621, 0.011767251873292396)), (1996, (0.4315608769582409, 0.5012184320516389, 0.06722069099012024)), (2016, (0.3435794575797364, 0.6208309236440906, 0.03558961877617299))]


In [7]:
state_text_path = "../states.txt"
states_save_path = "../by_state"

f = open(state_text_path)
states = f.readlines()
for state in states:
    print
    state = state.strip()
    staterdd = formatted.filter(lambda x: x[0][0] == state).map(lambda x: (x[0][1], x[1])).coalesce(1)
    staterdd.saveAsTextFile(states_save_path + "/" + state)